# Naïve Compression of DNA Sequences

## Quantification of Information

Information theory is based on the observation that knowing that a likely event has occurred is less informative than knowing that an unlikely event has occurred.

A quantification of information should have the following properties:
- Likely events should have a low information content, and events that are certain to occur should have no information content at all. Less likely events should have a higher information content.
- Independent events should have additive information content.

The self-information of an event $x$ is hence defined as

$$I(x)=-\log{}P(x).$$

By using the base-2 logarithm, the unit of self-information is bit. Hence, one bit is the amount of information gained by observing an event of probability $\frac{1}{2}$.

Self-information deals only with a single event $x$. By computing the expectation of the self-information with respect to the entire probability distribution $P(\text{x})$ we obtain the entropy

$$H(\text{x})=\mathbb{E}_{\text{x}\sim{}P}[I(\text{x}=x)]=-\mathbb{E}_{\text{x}\sim{}P}[\log{}P(\text{x}=x)]=-\sum_{x}P(x)\log{}P(x).$$

The entropy gives the average information that is expected in an event $x$ drawn from probability distribution $P(\text{x})$.

**Experiment** &mdash; Compute the entropy of the sequences `AAAA`, `AACC`, `ACGT`.

In [37]:
import entropy

for sequence in ["AAAA", "AACC", "ACGT"]:
    eta = entropy.entropy(sequence)
    print(f"Entropy of '{sequence}': {round(eta, 2):.2f} bit/symbol")

Entropy of 'AAAA': 0.00 bit/symbol
Entropy of 'AACC': 1.00 bit/symbol
Entropy of 'ACGT': 2.00 bit/symbol


## The FASTQ Format

The FASTQ format is the de-facto standard for the storage of reads, i.e., nucleotide sequences, including corresponding quality scores.

Each read is represented by a single FASTQ record, which consists of four lines:
- The first line contains the read identifier. It starts with `@`. Typically, sequencing machine vendors generate read identifiers in a proprietary systematic way.
- The second line contains the nucleotide sequence, where each nucleotide is represented with a single ASCII character.
- The third line starts with `+` and contains an optional description. Usually this line is left empty; it then only contains `+` as separator between the nucleotide sequence and the quality scores.
- The fourth line contains the quality scores. A quality score is a value indicating the confidence in a base call.

We can convert a FASTQ record (four lines) into a dictionary with the following function:

In [38]:
def fastq_lines_to_dict(lines):
    keys = ["id", "seq", "desc", "qual"]
    return {key: value for key, value in zip(keys, lines)}

**Experiment** &mdash; Parse the FASTQ file `example.fastq`.

In [39]:
file_name = "example.fastq"
records = []

with open(file_name, "r") as f:
    lines = []
    for line in f:
        lines.append(line.rstrip())
        if (len(lines)) == 4:
            records.append(fastq_lines_to_dict(lines))
            lines = []

for i, r in enumerate(records):
    print(f"Record {i}: {str(r)}")

Record 0: {'id': '@id0', 'seq': 'GATTTGGGGTTCAAAGCAGTATCGATCAAATA', 'desc': '+', 'qual': "!''*((((***+))%%%++)(%%%%).1***-"}
Record 1: {'id': '@id1', 'seq': 'GATTTGGGGTTCAAAGCAGTATCGATCAAATA', 'desc': '+', 'qual': "!''*((((***+))%%%++)(%%%%).1***-"}
Record 2: {'id': '@id2', 'seq': 'GATTTGGGGTTCAAAGCAGTATCGATCAAATA', 'desc': '+', 'qual': "!''*((((***+))%%%++)(%%%%).1***-"}
Record 3: {'id': '@id3', 'seq': 'GATTTGGGGTTCAAAGCAGTATCGATCAAATA', 'desc': '+', 'qual': "!''*((((***+))%%%++)(%%%%).1***-"}


## Compression of Nucleotide Sequences

**Experiment** &mdash; Concatenate all nucleotide sequences from the FASTQ file `example.fastq`. Compute the entropy and the maximum (worst-case) compressed size in bit and byte.

In [40]:
import math

seq = ""
for r in records:
    seq += r["seq"]
seq_len = len(seq)
print(f"Concatenated sequence length: {seq_len}")

eta = entropy.entropy(seq)
print(f"Entropy: {round(eta, 2):.2f} bit/symbol")

max_size_in_bit = math.ceil(eta * seq_len)
max_size_in_byte = math.ceil(max_size_in_bit / 8)

print(f"Maximum compressed size: {max_size_in_bit} bit \u2259 {max_size_in_byte} byte")
print(f"Worst-case compression ratio: {seq_len / max_size_in_byte:.1f}x")

Concatenated sequence length: 128
Entropy: 1.92 bit/symbol
Maximum compressed size: 246 bit ≙ 31 byte
Worst-case compression ratio: 4.1x


**Experiment** &mdash; Use gzip to beat the estimated worst-case compression.

In [41]:
import gzip

amplification_factor = 10
seq *= amplification_factor
seq_len = len(seq)

compressed_seq = gzip.compress(data=bytes(seq, "utf-8"))
decompressed_seq = gzip.decompress(data=compressed_seq).decode("utf-8")

if decompressed_seq != seq:
    raise RuntimeError(f"decompressed sequence is *not* equal to the original sequence")

max_size_in_bit = math.ceil(eta * seq_len)
max_size_in_byte = math.ceil(max_size_in_bit / 8)

print(f"Worst-case compression ratio: {seq_len / max_size_in_byte:.1f}x")
print(f"Gzip compression ratio: {seq_len / len(compressed_seq):.1f}x")

Worst-case compression ratio: 4.2x
Gzip compression ratio: 22.9x
